In [ ]:
import requests
import pandas as pd
import numpy as np
import sqlite3
import pymongo
import dns
import darksky_soccer as ds

%load_ext autoreload
%autoreload 2

### Pull match data from sqlite database and combine with darksky weather data

In [ ]:
darksky_key = ds.get_api_key('/Users/bmcgarry/.secret/darksky.txt')

match_stats = ds.match_rain_data('database.sqlite', 2013, darksky_key)

### Create aggregate match stats with either pandas or sqlite

In [ ]:
#Aggregate with Pandas
# team_stats = ds.calculate_aggregate_stats_pandas(match_stats)

#Aggregate with SQLite
team_stats = ds.calculate_aggregate_stats_sqlite(match_stats, 'database.sqlite')

### Insert data into atlas instance

In [ ]:
atlas_user = 'brian_m'
atlas_key = ds.get_api_key('/Users/bmcgarry/.secret/atlas.txt')
cluster_name = 'chocolate-clusters-so6jw'
collection_name = 'soccer_stats'

ds.insert_to_atlas(atlas_user,
                   atlas_key,
                   cluster_name,
                   collection_name,
                   team_stats)

### Pull match data from atlas

In [ ]:
client = pymongo.MongoClient(f'mongodb+srv://{atlas_user}:{atlas_key}@{cluster_name}.mongodb.net/test?retryWrites=true&w=majority')
db = client.test
collection = db[collection_name]

pd.DataFrame(list(collection.find({'season': 2011})))